In [1]:
import pandas as pd
import json
import os


In [2]:
json_files = os.listdir('StreamingHistory/')
combined_json = []

for file in json_files:
    with open(f"StreamingHistory/{file}", 'r') as f:
        data = json.load(f)
        combined_json.extend(data)

df = pd.DataFrame(combined_json)
df.to_csv('all_stream_history.csv', index=False)


In [3]:
df.head(1)


,ts,username,platform,ms_played,conn_country,ip_addr_decrypted,user_agent_decrypted,master_metadata_track_name,master_metadata_album_artist_name,master_metadata_album_album_name,...,episode_name,episode_show_name,spotify_episode_uri,reason_start,reason_end,shuffle,skipped,offline,offline_timestamp,incognito_mode
0,2012-04-26T04:00:05Z,1227567202,Windows 7 (Unknown Ed) SP0 [x86 0],207476,US,131.247.32.107,None,Good Girl,Carrie Underwood,Good Girl,...,None,None,None,popup,endplay,False,False,False,0,False


In [4]:
df = df.rename(columns={
    'ts': 'timestamp',
    'master_metadata_track_name': 'track_name',
    'master_metadata_album_artist_name': 'artist_name',
    'master_metadata_album_album_name': 'album_name',
    'spotify_track_uri': 'track_uri'
})


In [5]:
columns_to_drop = ["username", "platform", "conn_country", "ip_addr_decrypted", "user_agent_decrypted", "episode_name", "episode_show_name",
"spotify_episode_uri", "reason_start", "reason_end", "shuffle", "skipped", "offline", "offline_timestamp", "incognito_mode"]

#"ms_played",


In [6]:
df.drop(columns=columns_to_drop, inplace=True)


In [7]:
df.to_csv('all_stream_history_2.csv', index=False)


In [8]:
data = pd.read_csv('all_stream_history_2.csv')


In [11]:
data.sample(5)


,timestamp,ms_played,track_name,artist_name,album_name,track_uri
31,2012-04-26T22:37:47Z,25445,Only If For A Night,Florence + The Machine,Ceremonials,spotify:track:3I4wuL82QeU3IR518TuV33
26374,2022-02-11T05:43:23Z,156910,Fever,Dua Lipa,Future Nostalgia,spotify:track:3F1P0QzdXtBz0MXy7KIO5w
74557,2014-02-14T01:03:42Z,188172,Non Non Non - Rob & Chris Remix Edit,Lolita Jolie,Non Non Non,spotify:track:2vU79PAaihCpCHtBYkbvMZ
48024,2017-10-24T14:12:05Z,210634,Lights - Single Version,Ellie Goulding,Halcyon Days,spotify:track:44906V2WC6k59u06A8CO9D
29130,2022-07-11T01:36:59Z,4876,Something in the Orange,Zach Bryan,Something in the Orange,spotify:track:3WMj8moIAXJhHsyLaqIIHI


In [28]:
tracks_to_remove = ['Deeper Sleep', 'The Early Morning Rain', 'Chicago', 'White Noise For Baby Sleep', 'White Noise For Baby Sleep']
data = data[~data['track_name'].isin(tracks_to_remove)]


In [29]:
data['track_name'].value_counts().head(10)


track_name
Lights - Single Version                                 432
Shake It Off                                            322
Kill Bill                                               136
Roar                                                    131
Let Me Go (with Alesso, Florida Georgia Line & watt)    123
Fast Car                                                118
Still into You                                          112
It Ain’t Me (with Selena Gomez)                         110
Snooze                                                  109
The Cure                                                108
Name: count, dtype: int64

In [30]:
unique_songs = data.drop_duplicates(subset=['track_name', 'artist_name'])
unique_songs = unique_songs.dropna()


In [31]:
remix_tracks = unique_songs[unique_songs['track_name'].str.contains('remix', case=False, na=False)]
unique_songs = unique_songs[~data['track_name'].str.contains('remix', case=False, na=False)]


/var/folders/t6/bfyvsvq54r1f1_xd_gjqncpr0000gn/T/ipykernel_4733/89475970.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  unique_songs = unique_songs[~data['track_name'].str.contains('remix', case=False, na=False)]


In [39]:
unique_songs = unique_songs[unique_songs['ms_played'] >= 180000]
unique_songs.to_csv('filtered_dataset.csv', index=False)


In [40]:
unique_songs.shape


(4076, 6)

In [41]:
unique_songs.sample(10)


,timestamp,ms_played,track_name,artist_name,album_name,track_uri
6895,2013-11-03T15:59:27Z,204800,Stop,Spice Girls,Spiceworld,spotify:track:3FteycP8CaXS1MhjcXekVT
15595,2014-01-23T13:07:22Z,317945,Girlfriend,Icona Pop,THIS IS... ICONA POP,spotify:track:6bC3Pyx5UiHG5jiml4IdTk
5586,2013-10-07T21:05:05Z,211786,Kobe,Chief Keef,Finally Rich,spotify:track:1RkOJtrrYNeF5DLVtx0CG4
5212,2013-09-29T04:39:47Z,211998,Toujou La ?,Jocelyne Labylle,An Ti Fanm Gwada,spotify:track:0qBKq1DdkA84pp01s3zLzt
41629,2017-07-08T23:33:11Z,232970,I'm Every Woman,Whitney Houston,The Bodyguard - Original Soundtrack Album,spotify:track:2eHj0klWkwRQuIrNlPpCPa
83589,2016-05-05T02:21:15Z,292400,It's Not Right But It's Okay,Whitney Houston,My Love Is Your Love,spotify:track:7861kBrZ5aw7o4NfggcyYW
75683,2014-02-27T21:24:24Z,305080,Wit Me (feat. Lil Wayne),T.I.,Wit Me (feat. Lil Wayne) - Single,spotify:track:4FMtPpDo4l4AJitaFxvaMV
42247,2017-07-13T03:04:45Z,211160,Nicety,Michel'le,Michel'le,spotify:track:1vw9MuorZMUxhsQBGsgyo1
44094,2017-08-07T01:37:49Z,1025730,Mine (feat. Drake),Beyoncé,BEYONCÉ [Platinum Edition],spotify:track:63FrXif0Pdu4NAPvTh87mw
42048,2017-07-11T16:06:58Z,203426,To The Moon,Miguel,All I Want Is You,spotify:track:4YMEYbRvgzVXJoKi53SSgU


In [46]:
billie_tracks = unique_songs[unique_songs['artist_name'] == 'Taylor Swift']


In [47]:
billie_tracks


,timestamp,ms_played,track_name,artist_name,album_name,track_uri
807,2012-07-06T00:25:53Z,257813,Sparks Fly,Taylor Swift,Speak Now,spotify:track:6d9IiDcFxtFVIvt9pCqyGH
808,2012-07-06T00:29:36Z,222597,You Belong With Me,Taylor Swift,Fearless,spotify:track:3GCL1PydwsLodcpv0Ll1ch
809,2012-07-06T00:33:29Z,233327,Mean,Taylor Swift,Speak Now,spotify:track:5yEPktRqvIhko5QFF3aBhQ
811,2012-07-06T00:39:24Z,224867,If This Was A Movie,Taylor Swift,If This Was A Movie,spotify:track:2i8r4AhrUsRDXYVgwAvc9B
812,2012-07-06T00:46:01Z,233359,Love Story,Taylor Swift,Fearless,spotify:track:1vrd6UOGamcKNGnSHJQlSt
...,...,...,...,...,...,...
70635,2023-07-09T18:56:15Z,278539,Last Kiss (Taylor's Version),Taylor Swift,Speak Now (Taylor's Version),spotify:track:59KOoHFcw5XfICnO57holu
70636,2023-07-09T19:01:33Z,317960,Long Live (Taylor's Version),Taylor Swift,Speak Now (Taylor's Version),spotify:track:4hqJ4bSlYJOXb6Z4SRmzxs
70638,2023-07-09T19:05:29Z,235775,Ours (Taylor’s Version),Taylor Swift,Speak Now (Taylor's Version),spotify:track:3yNJkriPzWjkkDAWHIAVUq
70644,2023-07-09T19:32:41Z,252385,When Emma Falls in Love (Taylor’s Version) (Fr...,Taylor Swift,Speak Now (Taylor's Version),spotify:track:0zo975x58DlXbZllWvfYhg
